<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [7]:
import pandas as pd
import pandas_profiling as pf
from scipy import stats
import seaborn as sns

In [8]:
path_raw = '../data/raw/'

df = pd.read_csv(path_raw + 'ecommerce.csv', encoding='latin1', sep=',')
df.shape

(541909, 8)

In [9]:
profile = pf.ProfileReport(df, title='Ecommerce - Profile Report')
profile.to_file('../documentation/Ecommerce - Profile Report.html')

Summarize dataset:   0%|          | 0/21 [00:00<?, ?it/s]

C:\Users\WAS00005\Anaconda3\lib\site-packages\scipy\stats\stats.py:4594: RuntimeWarning:overflow encountered in longlong_scalars


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

**Tratamento dos dados**

Objetivo: Tratar tipos das variáveis, normalizá-las e remover valores extremos

Informações em: ".../documentation/Variaveis.xlsx"

In [29]:
#InvoiceStatus: Caso InvoiceNo comece com "C", o status será Cancelled, se não Done
df['InvoiceStatus'] = df['InvoiceNo'].apply(lambda x: 'Cancelled' if x.startswith('C') else 'Done')

#Fillnan
df['Description'].fillna('', inplace=True)
df['CustomerID'].fillna('', inplace=True)

#Flag para indicar presença de valores extremos em Quantity
df['QuantityOutlier'] = False

#Pq alguns pedidos com status Done possuem Quantidade menor ou igual a zero?
# Caso 1 - Todos possuem UnitPrice = 0 e Country = United Kingdom e InvoiceStatus = Done e CustomerID vazio
df.loc[(df['InvoiceStatus'] == 'Done') & (df['Quantity'] <= 0), 'QuantityOutlier'] = True

#QuantityZScore
df['QuantityZScore'] = stats.zscore(df['Quantity'])
#QuantityOutlierZScore
df['QuantityOutlierZScore'] = False
df.loc[(df['QuantityZScore'] > 3) | (df['QuantityZScore'] < -3), 'QuantityOutlierZScore'] = True

#UnitPriceZScore
df['UnitPriceZScore'] = stats.zscore(df['UnitPrice'])
#UnitPriceOutlierZScore
df['UnitPriceOutlierZScore'] = False
df.loc[(df['UnitPriceZScore'] > 3) | (df['UnitPriceZScore'] < -3), 'UnitPriceOutlierZScore'] = True

#Cast datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

Com o propósito de atingir nosso objetivo, serão desconsiderados os valores negativos sem investigar os motivos de sua existência

In [25]:
p_negativos = (df.query('UnitPrice <= 0').shape[0] / df.query('UnitPrice > 0').shape[0]) * 100
'Os registros com UnitPrice negativo representam {0} % da base'.format(p_negativos)

'Os registros com UnitPrice negativo representam 0.4666365092548647 % da base'

In [26]:
df = df.query('UnitPrice > 0').copy()

Desconsiderando outliers em UnitPrice e Quantity

In [42]:
df = df.query('QuantityOutlierZScore == False and UnitPriceOutlierZScore == False').copy()

In [45]:
df.to_pickle('../data/prep/dataset_ecommerce.pkl')